Since all of our file types are .txt, we will convert then into .csv and delete the .txt files before we push them into our database

In [ ]:
DATABASE_PATH = 'data/MarketHistoricalData.db'

In [ ]:
import csv, pathlib, os

def csv_converter(dir_path, delimiter=','):
    """Converts all .txt files within given dir to .csv and removes original"""
    
    # Converting 
    for path in pathlib.Path(dir_path).glob("*.txt"):
        with path.open() as txtfile, path.with_suffix(".csv").open(mode="w") as csvfile:
            reader = csv.reader(txtfile, delimiter = delimiter)
            writer = csv.writer(csvfile)
            for row in reader:
                writer.writerow(row)
    # Deleting 
    test = os.listdir(dir_path)
    for item in test:
        if item.endswith(".txt"):
            os.remove(os.path.join(dir_path, item))

# csv_converter("nasdaq_stocks/")
# csv_converter("nyse_stocks/")

The ticker file has a variable amount of spaces between ticker and company name, so we have to remove the spaces and place a comma there instead before we convert

In [ ]:

def fix_spacing(file_path):
    new_format = []
    with open(file_path, "r") as f:
        for line in f:
            new_format.append(",".join(line.split(maxsplit=1)))

    with open(file_path, 'w') as f:
        f.write("\n".join(new_format))

# fix_spacing("nasdaq_stock_tickers.txt")
# fix_spacing("nyse_stock_tickers.txt")
# csv_converter(".") #convert into csv

We will now create tables for all of our stocks 

In [ ]:
import pandas as pd 
new_names = {'<TICKER>' : 'Ticker', '<NAME>' : 'Name'}

nasdaq_tickers = pd.read_csv(
    'data/nasdaq_stock_tickers.csv').rename(columns=new_names)

nyse_tickers = pd.read_csv(
    'data/nyse_stock_tickers.csv').rename(columns=new_names)

print(nasdaq_tickers.head())
print(nyse_tickers.head())

In [ ]:
import os

def find_blanks(tickers, file_path):
    for ticker in tickers.loc[:, 'Ticker']:
        try:
            ticker_df = pd.read_csv(f'{file_path}{ticker.lower()}.csv')
            ticker_df.rename(
                columns=(
                lambda column_name : column_name.replace("<", "").replace(">", "").title()
                ),
                inplace= True
            )
            ticker_df.to_csv(f'{file_path}{ticker.lower()}.csv', index=False)
        except:
            try:
                os.remove(f'{file_path}{ticker.lower()}.csv')
                print(f"No data in {ticker}")
            except:
                print(f"{ticker} file doesn't exist")

# find_blanks(nasdaq_tickers, 'data/nasdaq_stocks/')
# find_blanks(nyse_tickers, 'data/nyse_stocks/')

Since the historical data for each stock comes in at differnt lengths, we first will find the smallest date in order to make all the time series the same length

In [ ]:
## Finding the lowest date 
from utils import _int_to_datetime
from datetime import datetime 

def find_lowest_date(tickers, file_path):

    lowest_date = datetime.now()
    lowest_ticker = ''

    for ticker in tickers.loc[:, 'Ticker']:
        ticker_df = pd.read_csv(f'{file_path}{ticker.lower()}.csv')
        date = _int_to_datetime(ticker_df.loc[0, '<DATE>'])

        if date < lowest_date:
            lowest_date = date
            lowest_ticker = ticker

    return lowest_date, lowest_ticker

# print(find_lowest_date(nasdaq_tickers, 'nasdaq_stocks/'))
# print(find_lowest_date(nyse_tickers, 'nyse_stocks/'))

So we have found that the lowest date on the nasdaq is (in YYYY-MM-DD) 1970-01-02 from 'AEP.US'
and on the NYSE is 1962-01-02 from 'GE.US'

We will now make all the time series the same length 

In [ ]:
ge_timeseries =pd.read_csv("data/nyse_stocks/ge.us.csv")

def normalize_dates(tickers, file_path):
    for ticker in tickers.loc[:, 'Ticker']:
        ticker_df = pd.read_csv(f'{file_path}{ticker.lower()}.csv')
        date = ticker_df.loc[0, '<DATE>']
        
        date_idx = (ge_timeseries['<DATE>'] == date).idxmax() # This will be -2 from the excel csv! 

        missing_dates = ge_timeseries.loc[:date_idx - 1, '<DATE>']
        insert_df = pd.DataFrame(columns=ge_timeseries.columns)
        insert_df['<DATE>'] = missing_dates
        insert_df['<TICKER>'] = ticker
        insert_df['<PER>'] = 'D'
        insert_df = insert_df.fillna(-1)

        normalized_df = pd.concat([insert_df, ticker_df], ignore_index=True)
        normalized_df.to_csv(f'{file_path}{ticker.lower()}.csv', index=False)

# normalize_dates(nasdaq_tickers, 'nasdaq_stocks/')
# normalize_dates(nyse_tickers, 'nyse_stocks/')

Pushing data into SQLite database

In [ ]:
import sqlite3
from utils import _ticker_to_table_name

conn = sqlite3.connect(DATABASE_PATH)
cursor = conn.cursor()


def push_table_to_db(tickers, file_path):

    for ticker in tickers.loc[:, 'Ticker']:
        ticker_df = pd.read_csv(f'{file_path}{ticker.lower()}.csv').rename(
            columns=(lambda x : x[1 : -1].title())
        )
        cursor.execute(
            f'''CREATE TABLE {_ticker_to_table_name(ticker)} (
                    Ticker text, 
                    Per text, 
                    Date text, 
                    Open real, 
                    High real, 
                    Low real, 
                    Close real, 
                    Vol integer, 
                    Openint integer
                    )'''
                    )
        ticker_df.to_sql(f'{_ticker_to_table_name(ticker)}' ,conn, if_exists='replace', index=False )

        

# push_table_to_db(nasdaq_tickers, 'nasdaq_stocks/')
# push_table_to_db(nyse_tickers, 'nyse_stocks/')

Let's compute some initial parameters 

In [ ]:
from momentum_strategy import QuantitativeMomentum

nasdaq_tickers = pd.read_csv('data/nasdaq_stock_tickers.csv')
nyse_tickers = pd.read_csv('data/nyse_stock_tickers.csv')

universe = pd.concat([nasdaq_tickers, nyse_tickers], ignore_index=True)

strategy = QuantitativeMomentum(
    'data/MarketHistoricalData.db',
    tickers=universe
)

# strategy.compute_parameters() # Using default parameters, uploads everything to SQL 


In [ ]:
[] == []

In [ ]:

query = f"""
        SELECT
        Return_{12}_Month,
        Percent_Positive_Over_{12}_Months, 
        Percent_Negative_Over_{12}_Months,
        Open
        FROM {_ticker_to_table_name('WOLF.US')} 
        WHERE Date >= {20081119}
        ORDER BY Date
        LIMIT 2"""

return_value = cursor.execute(query).fetchall()
return_value[0][0]

In [64]:
import heapq
capital = 100_000
universe_size = len(universe)
generic_momentum_size = int(universe_size * 0.1)
generic_momentum_screen = []

def _fip_score(perc_return: float, perc_pos_days: float, perc_neg_days: float) -> float:
    return perc_return * (perc_pos_days - perc_neg_days)

# Top 10% of generic momentum tickers
for ticker in universe['Ticker']:
    query = f"""SELECT 
    Return_{12}_Month,
    Percent_Positive_Over_{12}_Months, 
    Percent_Negative_Over_{12}_Months,
    Open
    FROM {_ticker_to_table_name(ticker)} 
    WHERE Date >= {20081119}
    ORDER BY Date
    LIMIT 2"""
    return_value = cursor.execute(query).fetchall()
    if return_value == []:
        pass
    elif return_value[0][0] == -1:
        pass
    else:
        generic_momentum, perc_pos, perc_neg, _ = return_value[0]
        next_open = return_value[1][3]
        heapq.heappush(generic_momentum_screen,
                        (generic_momentum,
                        _fip_score(
                            generic_momentum, perc_pos, perc_neg),
                            next_open,
                            ticker))
        if len(generic_momentum_screen) > generic_momentum_size:
            heapq.heappop(generic_momentum_screen)

# Choose top firms by fip_score next
top_firms = []
capital_available = (1/50) * capital
for generic_momentum, fip_score, next_open, ticker in generic_momentum_screen:
    shares_purchased = int( capital_available / (1.01 * next_open))
    if shares_purchased == 0:
        pass
    elif len(top_firms) < 50:
        heapq.heappush(top_firms, (fip_score, shares_purchased, 1.01 * next_open, ticker))
    else:
        heapq.heappushpop(top_firms, (fip_score, shares_purchased, 1.01 * next_open, ticker))

arr = [(ticker, shares_purcahsed, close) for fip_score, shares_purcahsed, close, ticker in top_firms]
result = sum(item[1] * item[2] for item in arr)
print(result)
arr

99708.82398910003


[('MDT.US', 82, 24.280198),
 ('UHT.US', 138, 14.467543),
 ('JEQ.US', 637, 3.1388073000000003),
 ('UPS.US', 55, 36.168806999999994),
 ('FCBC.US', 113, 17.564001),
 ('RTX.US', 97, 20.494415),
 ('FC.US', 452, 4.4238),
 ('RNR.US', 55, 36.152546),
 ('POOL.US', 168, 11.854875000000002),
 ('NRIM.US', 234, 8.5354191),
 ('COKE.US', 58, 34.040434000000005),
 ('CFFN.US', 232, 8.591605399999999),
 ('CNP.US', 273, 7.3227323),
 ('LUV.US', 245, 8.152922),
 ('GUT.US', 939, 2.1284942),
 ('JKHY.US', 143, 13.956987999999999),
 ('WWW.US', 269, 7.4226415),
 ('QCOM.US', 91, 21.974065),
 ('NTCT.US', 311, 6.4135),
 ('ES.US', 131, 15.232315),
 ('LNT.US', 199, 10.037511299999998),
 ('SPH.US', 208, 9.5755878),
 ('FDX.US', 37, 53.822395),
 ('PEP.US', 54, 36.736528),
 ('FNB.US', 308, 6.4838061),
 ('PZZA.US', 310, 6.4309225),
 ('JPM.US', 99, 20.201515),
 ('SHOO.US', 738, 2.7081736000000003),
 ('NATI.US', 191, 10.463802000000001),
 ('WAFD.US', 183, 10.902243),
 ('ORLY.US', 88, 22.533099999999997),
 ('ELS.US', 380, 5